In [0]:
from pyspark.sql.functions import col, sum, count, avg, round, desc


df_pedidos = spark.table("classes.silver.pedidos")


gold_geo = df_pedidos.groupBy("state") \
    .agg(
        sum("total_line_value").alias("receita_total"),
        count("order_id").alias("volume_pedidos"),
        round(avg("total_line_value"), 2).alias("ticket_medio")
    ) \
    .orderBy(col("receita_total").desc())


gold_geo = gold_geo.withColumn("receita_total", round(col("receita_total"), 2))



gold_produtos = df_pedidos.groupBy("product_group", "product_name") \
    .agg(
        sum("product_quantity").alias("qtd_vendida"),
        sum("total_line_value").alias("receita_gerada")
    ) \
    .orderBy(col("receita_gerada").desc())

gold_produtos = gold_produtos.withColumn("receita_gerada", round(col("receita_gerada"), 2))



gold_status = df_pedidos.groupBy("order_status") \
    .agg(
        count("order_id").alias("total_pedidos"),
        sum("total_line_value").alias("valor_em_jogo")
    ) \
    .withColumn("valor_em_jogo", round(col("valor_em_jogo"), 2)) \
    .orderBy(col("total_pedidos").desc())



print("--- Vendas por Estado (Top 5) ---")
display(gold_geo)

print("--- Mix de Produtos ---")
display(gold_produtos)

print("--- Status dos Pedidos ---")
display(gold_status)



In [0]:
%sql
SELECT * FROM classes.gold.vendas_por_estado ORDER BY receita_total DESC

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT * FROM classes.gold.mix_produtos ORDER BY receita_gerada DESC LIMIT 10

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT * FROM classes.gold.funil_vendas

Databricks visualization. Run in Databricks to view.

In [0]:

gold_geo.write.mode("overwrite").saveAsTable("classes.gold.vendas_por_estado")

gold_produtos.write.mode("overwrite").saveAsTable("classes.gold.mix_produtos")

gold_status.write.mode("overwrite").saveAsTable("classes.gold.funil_vendas")

print("Tabelas Gold salvas com sucesso!")